When we cannot afford to sample the quantity of interest many times at every design within an optimization, we can use surrogate models instead. Here we will show you how to use third party surrogates as well as the polynomial chaos surrogate provided with horsetail matching.

We will use the effective-quadratures package [Seshadri, P. and Parks, G. (2017) Effective-Quadratures (EQ): Polynomials for Computational Engineering Studies, The Open Journal, http://dx.doi.org/10.21105/joss.0016], (also see http://www.effective-quadratures.org/). We will also use pyKriging [pyKriging 0.5281/zenodo.593877] (also see http://pykriging.com/).

The HorstailMaching object takes a "surrogate" argument, which should be a function that takes an array of values of the uncertain parameters of size (num_points, num_uncertainties), and an array of the quantity of interest evaluated at these values of size (num_points) that returns a function that predicts the function output at any value of the uncertainties. The following examples should make this more clear.

Let's start with pyKriging...

In [1]:
from horsetailmatching import HorsetailMatching, UncertainParameter
from horsetailmatching.demoproblems import TP1, TP2


We use the pyKriging samplingplan function to give us 20 points found via latin hypercube sampling at which to evaluate the metric to create the surrogate. Then we create a function in the form required by horsetail matching called mySurrogate, and pass this as the surrogate argument when making the horestail matching object, along with the LHS points as the u_quadrature_points argument. 

In [20]:
from pyKriging.krige import kriging
from pyKriging.samplingplan import samplingplan

uparams = [UncertainParameter('uniform'), UncertainParameter('uniform')]

sp = samplingplan(2)
u_sampling = sp.optimallhc(25)

def mySurrogate(u_lhc, q_lhc):
    krig = kriging(u_lhc, q_lhc)
    krig.train()
    return krig.predict

theHM = HorsetailMatching(TP1, uparams, surrogate=mySurrogate, u_quadrature_points=u_sampling)
print('Metric evaluated with kriging surrogate: ', theHM.evalMetric([0, 1]))
theHM.surrogate = None
print('Metric evaluated with direct sampling: ', theHM.evalMetric([0, 1]))

('Metric evaluated with kriging surrogate: ', 15.508547313523406)
('Metric evaluated with direct sampling: ', 15.592935162751704)


Now we do a similar thing with the effective quadrature toolbox to make a quadratic polynomial surrogate. 



In [27]:
from equadratures import Polyreg
import numpy as np

uparams = [UncertainParameter('uniform'), UncertainParameter('uniform')]

U1, U2 = np.meshgrid(np.linspace(-1, 1, 5), np.linspace(-1, 1, 5))
u_tensor = np.vstack([U1.flatten(), U2.flatten()]).T

def mySurrogate(u_tensor, q_tensor):
    poly = Polyreg(np.mat(u_tensor), np.mat(q_tensor).T, 'quadratic')
    def model(u):
        return poly.testPolynomial(np.mat(u))
    return model

theHM = HorsetailMatching(TP1, uparams, surrogate=mySurrogate, u_quadrature_points=u_tensor)
print('Metric evaluated with quadratic surrogate: ', theHM.evalMetric([0, 1]))
theHM.surrogate = None
print('Metric evaluated with direct sampling: ', theHM.evalMetric([0, 1]))

('Metric evaluated with quadratic surrogate: ', 15.785982279490263)
('Metric evaluated with direct sampling: ', 15.699129346365819)


The polynomial chaos expansion used by the PolySurrogate class uses specific quadrature points over the uncertainty space to perform efficient integration, and so we must tell the HorsetailMatching object that these are the points at which to evaluate the quantity of interest when making the surrogate. This is done with he u_quadrature_points argument. 

In [29]:
from horsetailmatching.surrogates import PolySurrogate

uparams = [UncertainParameter('uniform'), UncertainParameter('uniform')]

thePoly = PolySurrogate(dimensions=len(uparams), order=4)
u_quadrature = thePoly.getQuadraturePoints()

def mySurrogate(u_quad, q_quad):
    thePoly.train(q_quad)
    return thePoly.predict

theHM = HorsetailMatching(TP1, uparams, surrogate=mySurrogate, u_quadrature_points=u_quadrature)
print('Metric evaluated with polynomial surrogate: ', theHM.evalMetric([0, 1]))
theHM.surrogate = None
print('Metric evaluated with direct sampling: ', theHM.evalMetric([0, 1]))

('Metric evaluated with polynomial surrogate: ', 15.712823802162529)
('Metric evaluated with direct sampling: ', 15.730374743597023)
